# Desktop Maps [More documentation to come!]

By Kenneth Burchfiel

Code released under the MIT License

(The maps created by this project are released into the public domain.)

This program will create various maps of the contiguous US that can serve as desktop backgrounds.

## Downloading shapefiles

I visited [the Census's shapefile website](https://www.census.gov/cgi-bin/geo/shapefiles/index.php) in order to download state, county, and zip-level boundaries. I chose to download 2021 boundaries (when available) rather than later versions so that Connecticut county boundaries would still be available.

I selected the following shapefiles for download. Items marked in **bold** are currently in use within this script.

1. Urban areas
2. Core-based statistical areas
3. Congressional Districts (the most recent ones available were from the 116th US Congress)
4. The US coastline
5. **Counties**
6. **Primary roads**
7. **States**
8. Zip Code Tabulation Areas (ZCTAs)

Once I had downloaded each of these datasets' corresponding zipfiles, I went ahead and extracted them, then placed them into a 'Datasets/Shapefiles' folder within my home folder. (Some of these datasets were well above the 100-megabyte limit for GitHub uploads, so I didn't add them directly into this project. However, these files can be downloaded for free using the web page linked above.)

In [1]:
from datetime import datetime
import time
start_time = time.time()
import pandas as pd
import geopandas as gpd
import folium
import time
from selenium import webdriver
import os
from PIL import Image # I ran conda install pillow
# in order to get PIL set up on my conda environment.

In [2]:
path_to_shapefiles = '../../../../../Datasets/Shapefiles/' # You'll need
# to update this path on your end to point to your own copies of the shapefiles.

Here are the the shapefile folders that I downloaded: 

(The shapefiles are present within, and share the same names as, these folders. For instance, the 'tl_2021_us_primaryroads.shp' shapefile is located within the 'tl_2021_us_primaryroads' folder.

In [3]:
os.listdir(path_to_shapefiles)

['tl_2021_us_primaryroads',
 'tl_2021_us_cbsa',
 'tl_2021_us_csa',
 'tl_2021_us_state',
 'tl_2021_us_cd116',
 'tl_2021_us_zcta520',
 'tl_2021_us_coastline',
 'tl_2020_us_uac20',
 'tl_2021_us_county']

## Reading in a polygon that provides a rough outline of the contiguous US:

(I created this file within geojson.io.)

In [4]:
gdf_contig_us_bounds = gpd.read_file('contiguous_us_bounds.json')
gdf_contig_us_bounds['geometry'] = gdf_contig_us_bounds['geometry'].to_crs('EPSG:4269')
gdf_contig_us_bounds['merge_key'] = 1
gdf_contig_us_bounds.rename(
    columns = {'geometry':'contig_us_bounds'}, inplace = True)
gdf_contig_us_bounds

,contig_us_bounds,merge_key
0,"POLYGON ((-125.96355 47.7781, -127.04332 47.30...",1


Note: I based the to_crs() call above on the following warning message I had received in an earlier version of the code:

UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4269
Right CRS: EPSG:4326

  gdf_roads['within_contig_us'] = gdf_roads['geometry'].within(

In [5]:
gdf_roads = gpd.read_file(
    path_to_shapefiles
    +'tl_2021_us_primaryroads/tl_2021_us_primaryroads.shp')
gdf_roads['geometry'] = gdf_roads['geometry'].simplify(tolerance = 0.001).copy()
gdf_roads['merge_key'] = 1
gdf_roads.query("RTTYP == 'I'", inplace = True) # Limits routes shown to
# US interstate highways
gdf_roads = gdf_roads.merge(gdf_contig_us_bounds, 
                            on = 'merge_key', how = 'left')
gdf_roads.drop('merge_key', axis = 1, inplace = True)
gdf_roads.head()

,LINEARID,FULLNAME,RTTYP,MTFCC,geometry,contig_us_bounds
0,1106073066667,W I- 20,I,S1100,"LINESTRING (-98.10768 32.61152, -98.07695 32.6...","POLYGON ((-125.96355 47.7781, -127.04332 47.30..."
1,110766972487,W I- 20,I,S1100,"LINESTRING (-102.76769 31.63855, -102.80296 31...","POLYGON ((-125.96355 47.7781, -127.04332 47.30..."
2,1104259293529,W I- 20,I,S1100,"LINESTRING (-103.3894 31.42685, -103.35124 31....","POLYGON ((-125.96355 47.7781, -127.04332 47.30..."
3,110206001621,W I- 20,I,S1100,"LINESTRING (-98.97076 32.37467, -99.01205 32.3...","POLYGON ((-125.96355 47.7781, -127.04332 47.30..."
4,110451633229,I- 785,I,S1100,"LINESTRING (-79.68662 36.0495, -79.6844 36.056...","POLYGON ((-125.96355 47.7781, -127.04332 47.30..."


## Cropping this list of roads to include only those within the contiguous US:

(This step ended up not being necessary, but I'm leaving this code in because it could serve as a useful reference for future projects that *do* require filtering results to ontly include those within a given boundary.)

In [6]:
gdf_roads['within_contig_us'] = gdf_roads['geometry'].within(
    gdf_roads['contig_us_bounds'])
gdf_roads.query("within_contig_us == True", inplace = True)
gdf_roads.drop('contig_us_bounds', axis = 1, inplace = True) # If this column
# were left in, we would end up with the following error message when trying
# to create a map:
# TypeError: Object of type Polygon is not JSON serializable
gdf_roads

,LINEARID,FULLNAME,RTTYP,MTFCC,geometry,within_contig_us
0,1106073066667,W I- 20,I,S1100,"LINESTRING (-98.10768 32.61152, -98.07695 32.6...",True
1,110766972487,W I- 20,I,S1100,"LINESTRING (-102.76769 31.63855, -102.80296 31...",True
2,1104259293529,W I- 20,I,S1100,"LINESTRING (-103.3894 31.42685, -103.35124 31....",True
3,110206001621,W I- 20,I,S1100,"LINESTRING (-98.97076 32.37467, -99.01205 32.3...",True
4,110451633229,I- 785,I,S1100,"LINESTRING (-79.68662 36.0495, -79.6844 36.056...",True
...,...,...,...,...,...,...
5603,11012815134136,I- 10 (Hov),I,S1100,"LINESTRING (-95.45164 29.7794, -95.45015 29.77...",True
5604,11013551329905,I- 25 (Express Lanes),I,S1100,"LINESTRING (-105.00229 39.76354, -104.99257 39...",True
5605,11013551330519,I- 25 (Express Lanes),I,S1100,"LINESTRING (-105.00545 39.76025, -105.00229 39...",True
5606,11013473368094,I- 25 (Express Lanes),I,S1100,"LINESTRING (-105.0057 39.76043, -105.00229 39....",True


## Importing US state boundaries:

In [7]:
gdf_states = gpd.read_file(path_to_shapefiles+'tl_2021_us_state/tl_2021_us_state.shp')
# I found that non-US states other than DC had a 'DIVISION' value of 0. I used that fact
# below to filter gdf_states to include only the 50 states plus DC:
# gdf_states[(gdf_states['STUSPS'].isin(['AK', 'HI']) == False) & (gdf_states["DIVISION"] != '0')]
gdf_states.query("DIVISION != '0' & STUSPS not in ['AK', 'HI']", inplace = True)
gdf_states.head()

,REGION,DIVISION,STATEFP,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,3,5,54,01779805,54,WV,West Virginia,00,G4000,A,62266298634,489204185,+38.6472854,-080.6183274,"POLYGON ((-80.85847 37.42831, -80.85856 37.428..."
1,3,5,12,00294478,12,FL,Florida,00,G4000,A,138961722096,45972570361,+28.3989775,-082.5143005,"MULTIPOLYGON (((-83.10874 24.62949, -83.10711 ..."
2,2,3,17,01779784,17,IL,Illinois,00,G4000,A,143778561906,6216493488,+40.1028754,-089.1526108,"POLYGON ((-89.17208 37.06831, -89.17296 37.067..."
3,2,4,27,00662849,27,MN,Minnesota,00,G4000,A,206232627084,18949394733,+46.3159573,-094.1996043,"POLYGON ((-92.74568 45.29604, -92.74629 45.295..."
4,3,5,24,01714934,24,MD,Maryland,00,G4000,A,25151992308,6979074857,+38.9466584,-076.6744939,"POLYGON ((-75.76659 39.37756, -75.7663 39.3738..."


## Creating an outline of the contiguous US:

We can accomplish this by 'dissolving' all of the lower 48 US states together within Geopandas.

In [8]:
gdf_us_outline = gdf_states.copy()
gdf_us_outline['US'] = 'US'
gdf_us_outline = gdf_us_outline[['US', 'geometry']].dissolve(
    by = 'US').reset_index()
# See https://geopandas.org/en/stable/docs/user_guide/aggregation_with_dissolve.html
gdf_us_outline

,US,geometry
0,US,"MULTIPOLYGON (((-93.83568 29.59677, -93.83649 ..."


Simplifying our state shapefiles:
(Note: I've found that dissolve() operations work best on pre-simplified datasets, which is why
this simplify() call is proceeding the dissolve() call.)

In [9]:
gdf_states['geometry'] = gdf_states['geometry'].simplify(tolerance=0.001).copy()
gdf_states.head()

,REGION,DIVISION,STATEFP,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,3,5,54,01779805,54,WV,West Virginia,00,G4000,A,62266298634,489204185,+38.6472854,-080.6183274,"POLYGON ((-80.84632 37.4234, -80.85946 37.4294..."
1,3,5,12,00294478,12,FL,Florida,00,G4000,A,138961722096,45972570361,+28.3989775,-082.5143005,"MULTIPOLYGON (((-83.10874 24.62949, -83.10352 ..."
2,2,3,17,01779784,17,IL,Illinois,00,G4000,A,143778561906,6216493488,+40.1028754,-089.1526108,"POLYGON ((-89.17208 37.06831, -89.17786 37.057..."
3,2,4,27,00662849,27,MN,Minnesota,00,G4000,A,206232627084,18949394733,+46.3159573,-094.1996043,"POLYGON ((-92.73547 45.30157, -92.75116 45.292..."
4,3,5,24,01714934,24,MD,Maryland,00,G4000,A,25151992308,6979074857,+38.9466584,-076.6744939,"POLYGON ((-75.78872 39.65076, -75.69367 38.460..."


## Importing boundaries for counties and county equivalents:

In [10]:
gdf_counties = gpd.read_file(
    path_to_shapefiles
    +'tl_2021_us_county/tl_2021_us_county.shp')
gdf_counties['geometry'] = gdf_counties['geometry'].simplify(tolerance = 0.001).copy()
# Determining color codes to use for our map:
# (These codes will simply alternate between 0 and 1 in order to create
# a checkerboard-like pattern within our county desktop map. 
# They don't have any informative value.)

In [11]:
gdf_counties['color_code'] = [i % 2 for i in range(len(gdf_counties))]
gdf_counties['map_color'] = gdf_counties['color_code'].map(
    {0:'black', 1:'white'})
gdf_counties.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,color_code,map_color
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,None,None,A,1477645345,10690204,+41.9158651,-096.7885168,"POLYGON ((-96.55551 42.08996, -96.55517 41.742...",0,black
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,None,None,None,A,680976231,61568965,+46.2946377,-123.4244583,"POLYGON ((-123.72656 46.38487, -123.25907 46.3...",1,white
2,35,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,None,None,None,A,6016818946,29090018,+34.3592729,-104.3686961,"POLYGON ((-104.44494 34.69166, -104.33972 34.6...",0,black
3,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,None,A,2169272970,22847034,+40.7835474,-096.6886584,"POLYGON ((-96.91094 41.04612, -96.46387 41.045...",1,white
4,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,None,None,None,A,1489645185,1718484,+40.1764918,-098.0468422,"POLYGON ((-98.27357 40.35036, -97.82082 40.350...",0,black


In [12]:
## Importing urban areas:

In [13]:
gdf_urban_areas = gpd.read_file(
    path_to_shapefiles
    +'tl_2020_us_uac20/tl_2020_us_uac20.shp')
# Calculating representative points for each of these urban areas:
gdf_urban_areas['geometry'] = (
    gdf_urban_areas['geometry'].simplify(tolerance = 0.001).copy())
gdf_urban_areas['central_coords'] = [
    [coord.y, coord.x] for coord in 
    gdf_urban_areas['geometry'].representative_point()]
# See https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.representative_point.html#geopandas.GeoSeries.representative_point 
# and https://geopandas.org/en/stable/docs/reference/geoseries.html 
# (for the use of .x and .y)
gdf_urban_areas.head()

,UACE20,GEOID20,NAME20,NAMELSAD20,LSAD20,MTFCC20,UATYP20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry,central_coords
0,67240,67240,"Pampa, TX","Pampa, TX Urban Area",67,G3500,U,S,21374659,0,+35.5451610,-100.9656498,"POLYGON ((-100.94507 35.54718, -100.94529 35.5...","[35.5476915, -100.96320837052134]"
1,23230,23230,"Delta, CO","Delta, CO Urban Area",67,G3500,U,S,16126572,129608,+38.7454236,-108.0619343,"POLYGON ((-108.04131 38.74094, -108.02754 38.7...","[38.745695, -108.05990631738334]"
2,36001,36001,"Gunnison, CO","Gunnison, CO Urban Area",67,G3500,U,S,9468678,11725,+38.5405505,-106.9384998,"POLYGON ((-106.91308 38.53804, -106.93582 38.5...","[38.5416945, -106.93485211530263]"
3,45775,45775,"Kuna, ID","Kuna, ID Urban Area",67,G3500,U,S,16384896,46009,+43.5010643,-116.4168990,"MULTIPOLYGON (((-116.39896 43.48833, -116.3974...","[43.4975285, -116.42118964627733]"
4,05410,05410,"Basalt, CO","Basalt, CO Urban Area",67,G3500,U,S,9433311,68797,+39.3835983,-107.0807354,"MULTIPOLYGON (((-107.1105 39.40463, -107.09949...","[39.383948000000004, -107.08345277860855]"


# Creating a black background for our maps:

Because I prefer dark-mode desktop themes, I wanted the background of my maps to be black. However, tileless Folium maps have a relatively light background color by default. Therefore, I decided to go to https://geojson.io and draw a giant rectangle around the contiguous US that I could then use as a dark background. This is a pretty 'janky' solution, but it works for the purposes of these maps!

(See [this post](https://github.com/python-visualization/branca/issues/91#issuecomment-1166392776) by aluthfian for a more sophisticated solution.

In [14]:
us_background = gpd.read_file('us_background.json')
us_background

,geometry
0,"POLYGON ((-47.00516 -6.2409, -47.00516 69.9314..."


## Creating a map of states and interstate highways

Note: Initially, I planned to tweak the size of the Chrome webdriver window so that the US map would take up an ideal proportion of the screenshot. However, I found that some Chrome image width and height settings failed to work correctly, even though certain lower *and* higher resolution settings worked just fine.

Therefore, I instead chose to use the Pillow library (imported as PIL) to crop the screenshot created by the Chrome webdriver. In order to preserve the Leaflet logo on the bottom right, I set the map's starting latitude and longitude coordinates so that the US would be centered near the bottom right of the image. That way, in order to create my centered image, I would only have to crop out the left and top areas of the image.

## Defining style functions:

These functions determine how various GeoJSON elements will be displayed within our desktop maps.

In [15]:
road_style_function = lambda feature: {
        "color": "orange",
        "opacity":1,
        "weight": 2,
        "fillOpacity":0
    }

state_style_function = lambda feature: {
        "color": "white",
        "opacity":1,
        "weight": 2,
        "fillOpacity":0
    }

us_outline_style_function = lambda feature: {
        "color": "white",
        "opacity":1,
        "weight": 2,
        "fillOpacity":0
    }


county_style_function = lambda feature: {
        "color": "orange",
        "opacity":1,
        "weight": 2,
        "fillOpacity":0}

urban_area_style_function = lambda feature: {
        "opacity":0,
        "fillColor": 'orange',
        "fillOpacity":1}

# The following alternative county style function creates more of a 
# 'camouflaged' look.
# county_style_function = lambda feature: {
#         "color": "white",
#         "opacity":0,
#         "weight": 0,
#         "fillOpacity":0.3,
#         "fillColor":feature['properties']['map_color']
#     }
# The fillColor entry within the above code was based on
# the example shown in
# https://python-visualization.github.io/folium/latest/user_guide/geojson/geojson.html .

background_style_function = lambda feature: {
    'fillColor':'000000', 'fillOpacity':1}

# Creating HTML maps:

## Mapping states and roads:

In [16]:

# Note: I shifted my regular contiguous US starting coordinates (38 and -95)
# to the north and west so that, when cropping the image, the Leaflet copyright
# notice would remain in place

m_states_and_roads = folium.Map([43, -107], 
               zoom_start = 7, tiles = None,
              zoom_control = False) # Since we're only interested in the
# static version of this map, there's no reason to include zoom control
# (which would clutter the screenshot)
folium.GeoJson(us_background, style_function = background_style_function).add_to(m_states_and_roads)
folium.GeoJson(gdf_states, style_function = state_style_function).add_to(m_states_and_roads)
folium.GeoJson(gdf_roads, style_function = road_style_function).add_to(m_states_and_roads)
m_states_and_roads.save('maps/states_and_roads.html')
# m_states_and_roads

## Mapping county outlines:

In [17]:
m_counties = folium.Map([43, -107], 
               zoom_start = 7, tiles = None,
              zoom_control = False)

folium.GeoJson(
    us_background, 
    style_function = background_style_function).add_to(m_counties)
folium.GeoJson(gdf_counties, style_function = county_style_function).add_to(m_counties)
# folium.GeoJson(gdf_states, style_function = state_style_function).add_to(m_counties)
# The following line was useful when county fill colors were displayed rather 
# than outlines. However, now that county outlines are present, they themselves
# form an outline of the country's boundaries--rendering this additional 
# US outline unnecessary.
# folium.GeoJson(gdf_us_outline,
              # style_function = us_outline_style_function).add_to(m_counties)

m_counties.save('maps/counties.html')
# m_counties

## Mapping urban area boundaries:

In [18]:
m_urban_areas = folium.Map([43, -107], 
               zoom_start = 7, tiles = None,
              zoom_control = False)

folium.GeoJson(
    us_background, 
    style_function = background_style_function).add_to(m_urban_areas)
folium.GeoJson(gdf_urban_areas, 
               style_function = urban_area_style_function).add_to(m_urban_areas)
# I prefer the look of this map without the US outline, but you can find
# a variant with this outline added in within the extra_maps folder.
# folium.GeoJson(gdf_us_outline,
#                style_function = us_outline_style_function).add_to(m_urban_areas)

m_urban_areas.save('maps/urban_areas.html')
# m_urban_areas

## Mapping urban areas' central points:

In [19]:
m_urban_area_dots = folium.Map([43, -107], 
               zoom_start = 7, tiles = None,
              zoom_control = False)
folium.GeoJson(
    us_background, 
    style_function = background_style_function).add_to(m_urban_area_dots)
# Adding central points to the map in the form of CircleMarker objects:
for i in range(len(gdf_urban_areas)):
    folium.CircleMarker(
        location = gdf_urban_areas.iloc[i]['central_coords'],
    stroke = False,
    fill_opacity=1,
    fill_color = 'orange',
    radius = 5).add_to(m_urban_area_dots)
folium.GeoJson(gdf_us_outline,
               style_function = us_outline_style_function).add_to(
    m_urban_area_dots)
# This code was based on:
# https://python-visualization.github.io/folium/latest/user_guide/vector_layers/circle_and_circle_marker.html

m_urban_area_dots.save('maps/urban_area_dots.html')

# m_urban_area_dots

# Creating screenshots of these maps:

## Creating a function that will create PNG copies of HTML maps:

(This function was based off code in the [choropleth_map_functions.py](https://github.com/kburchfiel/pfn/blob/main/Mapping/choropleth_map_functions.py) file within the Mapping section of my [Python for Nonprofits](https://github.com/kburchfiel/pfn/tree/main) project.

In [20]:
def create_map_screenshot(html_map_folder,
    map_filename, png_map_folder, 
    delete_html_file):
    print("Generating screenshot.")
    options = webdriver.ChromeOptions()
    # Source: https://www.selenium.dev/documentation/webdriver/browsers/chrome/
    options.add_argument(f'--window-size={driver_window_width},\
{driver_window_height}') # I found that this window
    # size, along with a starting zoom of 6 within our mapping code,
    # created a relatively detailed map of the contiguous 48 US states. 
    # If you'd like to create an even more detailed map, consider setting 
    # your starting zoom to 7 and your window size to 6000,3375.
    options.add_argument('--headless') # In my experience, this addition 
    # (which prevents the Selenium-driven browser from displaying on your 
    # computer) was necessary for allowing 4K screenshots to get saved
    # as 3840x2160-pixel images. Without this line, the screenshots would 
    # get rendered with a resolution of 3814x1868 pixels.
    # Source of the above two lines:  
    # https://www.selenium.dev/documentation/webdriver/browsers/chrome/
    # and
    # https://github.com/GoogleChrome/chrome-launcher/blob/main/docs/chrome-flags-for-tools.md
    # I learned about the necessity of using headless mode *somewhere* on 
    # StackOverflow. Many answers to this question regarding generating 
    # screenshots reference it as an important step, for instance:
    # https://stackoverflow.com/questions/41721734/take-screenshot-of-full-page-with-selenium-python-with-chromedriver/57338909

    
    # Launching the Selenium driver:
    driver = webdriver.Chrome(options=options) 
    # Source: https://www.selenium.dev/documentation/webdriver/browsers/chrome/
    
    # Navigating to the map:
    # Note: I needed to precede the local path with 'file://' as 
    # noted by GitHub user lukeis here: 
    # https://github.com/seleniumhq/selenium-google-code-issue-archive/issues/3997#issuecomment-192014472
    map_path = f"file://{html_map_folder}/{map_filename}.html"
    print(map_path)
    driver.get(map_path)
    # Source: https://www.selenium.dev/documentation/
    time.sleep(1) # Helps ensure that the browser has enough 
    # time to download
    # map contents from the tile provider. This time might need to be
    # increased if a slow internet connection is in use. Conversely,
    # if no tiles are being incorporated into the map, 
    # there may not be any need to call
    # time.sleep().
    # Taking our screenshot and then saving it as a PNG image:
    driver.get_screenshot_as_file(f"{png_map_folder}/{map_filename}.png")
    # Source: 
    # https://selenium-python.readthedocs.io/api.html#selenium.webdriver.remote.webdriver.WebDriver.get_screenshot_as_file
    
    # Exiting out of the webdriver:
    driver.quit()
    # Source: https://www.selenium.dev/documentation/
    
    if (delete_html_file == True):
        os.remove(f"{html_map_folder}/{map_filename}.html")
        print("Removed HTML copy of map.")

### Configuring the Selenium Webdriver that will be used to generate these screenhsots:

In [21]:
uhd_width = 3840
uhd_height = 2160

scale_factor = 2

driver_window_width = uhd_width * scale_factor
driver_window_height = uhd_height * scale_factor
driver_window_width, driver_window_height

# Note: an uhd_width of 3840, an uhd_height of 2160,
# a scale factor of 3, and a folium.Map() starting_zoom of 8
# work pretty nicely together, but the resulting 11520*6480 resolution is
# probably overkill for just about any end user.

# As discussed earlier, I had originally planned to tweak these width and 
# height settings in order to produce a a cropped map, but I ended up with
# a severely truncated output. Therefore, I instead decided to 
# use PIL to complete the cropping tasks.

(7680, 4320)

In [22]:
map_folder = os.getcwd() + '/maps'

## Creating a function for cropping these screenshots:

In [23]:
def crop_screenshot(path_to_source, path_to_dest,
                   left, upper, right, lower, dest_file_format = 'PNG'):
    '''
    This function crops an image located at path_to_source, then 
    saves it to path_to_dest. 
    
    (These paths can be either relative or 
    absolute. To overwrite the original image, simply make path_to_dest
    the same value as path_to_source. Make sure to include the file extension
    within these paths as well.)

    dest_file_format: the extension to use when saving the image. It should
    match that shown within path_to_dest.

    left, upper, right, lower: the values to pass to PIL's crop() function.
    For more details on these values, see:
    https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.crop
    
    '''
    
    # The following code was based on:
    # https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.crop
    
    with Image.open(path_to_source) as im:
        im_crop = im.crop((left, upper, right, lower))
    
    # Overwriting the original image (since there's no reason to keep it):
    
    im_crop.save(path_to_dest, dest_file_format)

### Setting parameters that will be passed to the crop_screenshot() function:

In [24]:
left_crop = 2000
top_crop = left_crop * 9/16 # Initializing top_crop as
# a scaled version of left_crop ensures that the final
# image will remain in a 16:9 aspect ratio.
right = 7680
lower = 4320
print(left_crop, top_crop, right, lower)

2000 1125.0 7680 4320


## Using create_map_screenshot() and crop_creenshot() to create screenhots of our maps, then deleting the HTML files on which they were based:

In [25]:
create_map_screenshot(
    html_map_folder = map_folder,
    map_filename = 'states_and_roads',
    png_map_folder = map_folder,
    delete_html_file = True)

crop_screenshot(
    'maps/states_and_roads.png', 'maps/states_and_roads.png',
    left = left_crop, upper = top_crop, right = right, lower = lower)

Generating screenshot.
file:///home/kjb3/kjb3docs/programming/py/kjb3_programs_2/desktop_maps/maps/states_and_roads.html
Removed HTML copy of map.


In [26]:
create_map_screenshot(
    html_map_folder = map_folder,
    map_filename = 'counties',
    png_map_folder = map_folder,
    delete_html_file = True)

crop_screenshot(
    'maps/counties.png', 'maps/counties.png',
    left = left_crop, upper = top_crop, 
    right = right, lower = lower)

Generating screenshot.
file:///home/kjb3/kjb3docs/programming/py/kjb3_programs_2/desktop_maps/maps/counties.html
Removed HTML copy of map.


In [27]:
create_map_screenshot(
    html_map_folder = map_folder,
    map_filename = 'urban_areas',
    png_map_folder = map_folder,
    delete_html_file = True)

crop_screenshot(
    'maps/urban_areas.png', 'maps/urban_areas.png',
    left = left_crop, upper = top_crop, 
    right = right, lower = lower)

Generating screenshot.
file:///home/kjb3/kjb3docs/programming/py/kjb3_programs_2/desktop_maps/maps/urban_areas.html
Removed HTML copy of map.


In [28]:
create_map_screenshot(
    html_map_folder = map_folder,
    map_filename = 'urban_area_dots',
    png_map_folder = map_folder,
    delete_html_file = True)

crop_screenshot(
    'maps/urban_area_dots.png', 'maps/urban_area_dots.png',
    left = left_crop, upper = top_crop, 
    right = right, lower = lower)

Generating screenshot.
file:///home/kjb3/kjb3docs/programming/py/kjb3_programs_2/desktop_maps/maps/urban_area_dots.html
Removed HTML copy of map.


In [29]:
current_timestamp = datetime.now(
).isoformat(sep = ' ', timespec = 'seconds')
# See https://docs.python.org/3/library/datetime.html#datetime.datetime.isoformat
current_time = time.ctime()
end_time = time.time(); run_time = end_time - start_time
print(f"{current_timestamp}: Finished running script in {round(run_time, 3)} seconds.") 

2024-08-27 01:24:48: Finished running script in 41.483 seconds.
